# Exploration Notebook
---
An exploration notebook for tree point clouds. We will compare and analyse isolated trees using three different point cloud sources:
- _[AHN](https://www.ahn.nl) data_ (publicly available)
- _Cyclo Media_
- _Sonarski_


**Visual Impressions**

This particular tree is located at 1219.13, 4874.34 (LTR: AHN, CycloMedia, Sonarski):

![Comparison of datasets (side-view)](../imgs/20_side.png)
![Comparison of datasets (top-view)](../imgs/20_top.png)

### 0. Setup Notebook
---

**Import Modules**

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from tqdm import tqdm
from shapely.geometry import Polygon
from scipy.spatial import ConvexHull

import utils.math_utils as math_utils
import utils.plot_utils as plot_utils
import utils.las_utils as las_utils
import utils.o3d_utils as o3d_utils
import utils.ahn_utils as ahn_utils
import preprocessing.ahn_preprocessing as ahn_preprocessing

**Set-up Logging**

In [ ]:
log.basicConfig(
    level=log.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        log.FileHandler("debug.log"),
        log.StreamHandler()
    ]
)

### 1. Load Data
---

**Load Tree Point Cloud** _(AHN / Cylco Media / Sonarski)_

In [ ]:
# Load point cloud data
source = 'cyclo'

if source == 'ahn':
    tree_cloud = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_AHN.las')
elif source == 'cyclo':
    tree_cloud = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_Cyclo.las')
else:
    tree_cloud = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_Sonarski.las')


**Visualize**

In [ ]:
o3d.visualization.draw_geometries([tree_cloud])

#### AHN Surface

In [ ]:
# Load the AHN point cloud.
# WARNING: this may require 16+ GB of RAM, depending on the size of the cloud.
ahn_cloud_file = '../datasets/C_25DN2.LAZ'
ahn_cloud = las_utils.read_las(ahn_cloud_file)

In [ ]:
in_folder = '../..'
ahn_data_folder = '../'

# Clip the corresponding AHN point cloud tiles and create the AHN ground surfaces, with a 0.1m resolution, and save them as an .npz file.
ahn_preprocessing.clip_ahn_las_folder(ahn_cloud, in_folder, out_folder=ahn_data_folder, buffer=4,
                        resume=False, hide_progress=False)

**Ground Extraction**

In [ ]:
# Create the reader for .npz data.
npz_reader = ahn_utils.NPZReader(ahn_data_folder)

In [ ]:
ahn_tile = npz_reader.filter_tile(tilecode)
fast_z = FastGridInterpolator(ahn_tile['x'], ahn_tile['y'],
                                ahn_tile['ground_surface'])